In [1]:
import pypdf

In [22]:
from IPython.display import Markdown

In [12]:
# pdfreader = pypdf.PdfReader()
file = open('./paper3.pdf', 'rb')
pdf = pypdf.PdfReader(file)

In [25]:
import pandas as pd

In [64]:
df = pd.read_excel('dataset.xlsx')

# df.head(2)

df1 = df.sample(frac=0.5).head(100)
# df1.shape

# df1.columns

colstodrop = ['Identifier (RIC)', 'Country of Exchange']
df1.drop(columns= colstodrop, inplace=True)

missingtable = (df1.isna().sum() > 0)
missingcols = missingtable[missingtable.values == True].index.tolist()
missingcols

['Total Assets\n(FY0, AUD)',
 'Net Profit Margin',
 'Return On Assets',
 'Return On Equity',
 'Total Revenue per Share\n(FY0, AUD)',
 'Liquidity Ten Day Volume',
 'WACC Equity Risk Premium, (%)',
 'Dividend Yield',
 'Long Term Debt To Equity']

In [65]:
df1[missingcols].head(2)

,"Total Assets\n(FY0, AUD)",Net Profit Margin,Return On Assets,Return On Equity,"Total Revenue per Share\n(FY0, AUD)",Liquidity Ten Day Volume,"WACC Equity Risk Premium, (%)",Dividend Yield,Long Term Debt To Equity
55,5.676670e+06,NaN,0.0,0.0,0.000049,NaN,0.033578,0.0,0.0
642,4.612045e+09,-10.19405,0.0,0.0,2.632255,4.917623e+06,0.033578,0.0,0.0


In [30]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [69]:
from sklearn.preprocessing import FunctionTransformer

In [47]:
from sklearn.impute import SimpleImputer

In [70]:
import numpy as np

In [71]:
log_transfomer = FunctionTransformer(np.log, validate=False)

In [74]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100 entries, 55 to 643
Data columns (total 12 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Company Name                        100 non-null    object 
 1   Total Assets
(FY0, AUD)             98 non-null     float64
 2   GICS Sector Name                    100 non-null    object 
 3   Net Profit Margin                   4 non-null      float64
 4   Return On Assets                    28 non-null     float64
 5   Return On Equity                    28 non-null     float64
 6   Total Revenue per Share
(FY0, AUD)  84 non-null     float64
 7   GICS Industry Name                  100 non-null    object 
 8   Liquidity Ten Day Volume            88 non-null     float64
 9   WACC Equity Risk Premium, (%)       99 non-null     float64
 10  Dividend Yield                      12 non-null     float64
 11  Long Term Debt To Equity            28 non-null  

In [86]:
df1['Company Name'].iloc[0]

'Australia United Mining Ltd'

In [88]:
for c in df1.columns:
    # print(c)
    print(f'Type : {type(df1[c].iloc[0])}')

Type : <class 'str'>
Type : <class 'numpy.float64'>
Type : <class 'str'>
Type : <class 'numpy.float64'>
Type : <class 'numpy.float64'>
Type : <class 'numpy.float64'>
Type : <class 'numpy.float64'>
Type : <class 'str'>
Type : <class 'numpy.float64'>
Type : <class 'numpy.float64'>
Type : <class 'numpy.float64'>
Type : <class 'numpy.float64'>


In [90]:
numcols = [col for col in df1.columns if (type(df1[col].iloc[0]) != str)]
numcols

['Total Assets\n(FY0, AUD)',
 'Net Profit Margin',
 'Return On Assets',
 'Return On Equity',
 'Total Revenue per Share\n(FY0, AUD)',
 'Liquidity Ten Day Volume',
 'WACC Equity Risk Premium, (%)',
 'Dividend Yield',
 'Long Term Debt To Equity']

In [92]:
df1[numcols].head(2)

,"Total Assets\n(FY0, AUD)",Net Profit Margin,Return On Assets,Return On Equity,"Total Revenue per Share\n(FY0, AUD)",Liquidity Ten Day Volume,"WACC Equity Risk Premium, (%)",Dividend Yield,Long Term Debt To Equity
55,5.676670e+06,NaN,0.0,0.0,0.000049,NaN,0.033578,0.0,0.0
642,4.612045e+09,-10.19405,0.0,0.0,2.632255,4.917623e+06,0.033578,0.0,0.0


In [150]:
bigtable = df1[numcols].max() > 1e4
bigcols = bigtable[bigtable == True].index.tolist()
bigcols

['Total Assets\n(FY0, AUD)', 'Net Profit Margin', 'Liquidity Ten Day Volume']

In [143]:
mset = set(missingcols)
bset = set(bigcols)

In [152]:
functransformer = FunctionTransformer(lambda x : 1/x, validate=False)
numericaltransformer = ColumnTransformer([
    ('numtransformer', SimpleImputer(strategy='mean',), missingcols),
    # ('logtransformer', functransformer, bigcols)
],remainder = 'passthrough')

logtransformer = ColumnTransformer([
    ('logtransformer', functransformer, bigcols)
], remainder = 'passthrough')

nonmissingcols = [col for col in df1.columns if col not in missingcols]
nonmissingcols

['Company Name', 'GICS Sector Name', 'GICS Industry Name']

In [160]:
a = pd.DataFrame(numericaltransformer.fit_transform(df1[missingcols]), columns=missingcols, index = df1.index)
b = pd.DataFrame(logtransformer.fit_transform(a[bigcols]), columns= bigcols, index = df1.index)
b.head(2)

,"Total Assets\n(FY0, AUD)",Net Profit Margin,Liquidity Ten Day Volume
55,1.761596e-07,-0.000133,2.304239e-06
642,2.168236e-10,-0.098096,2.033503e-07


In [154]:
a.head(2)

,"Total Assets\n(FY0, AUD)",Net Profit Margin,Return On Assets,Return On Equity,"Total Revenue per Share\n(FY0, AUD)",Liquidity Ten Day Volume,"WACC Equity Risk Premium, (%)",Dividend Yield,Long Term Debt To Equity
0,5.676670e+06,-7514.356296,0.0,0.0,0.000049,4.339828e+05,0.033578,0.0,0.0
1,4.612045e+09,-10.194050,0.0,0.0,2.632255,4.917623e+06,0.033578,0.0,0.0


In [128]:
remcols = list(mset.difference(bset))
remcols

['Return On Assets',
 'Dividend Yield',
 'Return On Equity',
 'Total Revenue per Share\n(FY0, AUD)',
 'Long Term Debt To Equity',
 'WACC Equity Risk Premium, (%)']

In [157]:
df1.columns

Index(['Company Name', 'Total Assets\n(FY0, AUD)', 'GICS Sector Name',
       'Net Profit Margin', 'Return On Assets', 'Return On Equity',
       'Total Revenue per Share\n(FY0, AUD)', 'GICS Industry Name',
       'Liquidity Ten Day Volume', 'WACC Equity Risk Premium, (%)',
       'Dividend Yield', 'Long Term Debt To Equity'],
      dtype='object')

In [162]:
a[remcols].join(b)

,Return On Assets,Dividend Yield,Return On Equity,"Total Revenue per Share\n(FY0, AUD)",Long Term Debt To Equity,"WACC Equity Risk Premium, (%)","Total Assets\n(FY0, AUD)",Net Profit Margin,Liquidity Ten Day Volume
55,0.000000,0.000000,0.000000,0.000049,0.000000,0.033578,1.761596e-07,-0.000133,2.304239e-06
642,0.000000,0.000000,0.000000,2.632255,0.000000,0.033578,2.168236e-10,-0.098096,2.033503e-07
654,-0.061836,0.027604,-0.873143,0.000001,0.046754,0.033578,2.541770e-08,-0.000133,2.304239e-06
220,-0.061836,0.000000,-0.873143,0.000939,0.046754,0.033578,5.041369e-07,-0.000133,1.866921e-05
727,-0.009200,0.027604,-0.009700,0.853931,0.000200,0.033578,1.863985e-09,-0.000133,1.207728e-06
...,...,...,...,...,...,...,...,...,...
771,-0.061836,0.000000,-0.873143,0.000214,0.046754,0.033578,4.354875e-08,-0.000133,9.627061e-06
583,0.000000,0.027604,0.000000,0.000040,0.000000,0.033578,1.840963e-07,-0.000133,2.329313e-06
576,-0.061836,0.017713,-0.873143,12.679816,0.046754,0.063560,1.492484e-10,0.057579,3.028344e-07
208,-0.061836,0.027604,-0.873143,0.000500,0.046754,0.033578,2.163635e-07,-0.000133,4.602577e-05


In [165]:
dffinal = df1[['Company Name', 'GICS Sector Name', 'GICS Industry Name']].join(a[remcols].join(b))
dffinal.head(2)

,Company Name,GICS Sector Name,GICS Industry Name,Return On Assets,Dividend Yield,Return On Equity,"Total Revenue per Share\n(FY0, AUD)",Long Term Debt To Equity,"WACC Equity Risk Premium, (%)","Total Assets\n(FY0, AUD)",Net Profit Margin,Liquidity Ten Day Volume
55,Australia United Mining Ltd,Materials,Metals & Mining,0.0,0.0,0.0,0.000049,0.0,0.033578,1.761596e-07,-0.000133,2.304239e-06
642,Sandfire Resources Ltd,Materials,Metals & Mining,0.0,0.0,0.0,2.632255,0.0,0.033578,2.168236e-10,-0.098096,2.033503e-07


In [67]:
dffinal = pd.DataFrame(numericaltransformer.fit_transform(df1[missingcols]), columns= missingcols, index = df1.index)
# dffinal = pd.concat([df1[nonmissingcols], dffinal], axis = 1)
dffinal.head(2)

,"Total Assets\n(FY0, AUD)",Net Profit Margin,Return On Assets,Return On Equity,"Total Revenue per Share\n(FY0, AUD)",Liquidity Ten Day Volume,"WACC Equity Risk Premium, (%)",Dividend Yield,Long Term Debt To Equity
55,5.676670e+06,-7514.356296,0.0,0.0,0.000049,4.339828e+05,0.033578,0.0,0.0
642,4.612045e+09,-10.194050,0.0,0.0,2.632255,4.917623e+06,0.033578,0.0,0.0


In [166]:
# dffinal = df1[nonmissingcols].join(dffinal,)
# dffinal.reset_index(drop = True, inplace =True)

dffinal.to_csv('./dataset.csv', index = False)

In [168]:
import os

In [173]:
import pathlib

In [180]:
os.path.normpath(pathlib.PurePosixPath('what/eh'))

'what/eh'

In [176]:
pathlib.Path(os.path.abspath('../../anne_rose_uk/anne_rose_CARE_part1.docx')).parent.as_posix()

'/home/auru/coding/projects/uk_projects/anne_rose_uk'

In [8]:
from langchain_community.document_loaders.odt import UnstructuredODTLoader
from langchain_community.document_loaders import DocusaurusLoader, Docx2txtLoader

In [ ]:
DocusaurusLoader()

In [1]:
from langchain_community.document_loaders import UnstructuredWordDocumentLoader

In [15]:
l = Docx2txtLoader('../../anne_rose_uk/care_part2/Anne_Rose_CARE_Part_2_specification.docx')

In [16]:
l.load()

[Document(page_content='March 2024\t\t \n\nChampionships at Rare Earth (CARE) part 2 – Implementation\n\nThe Assignment - Part 2\n\n\t\tProduce a IntelliJ project implementing the specified CARE interface by completing tasks described below.  Your project must be capable of running correctly using the software in UH labs.\n\n\t\t\n\n\t\tYour CARE project should display the qualities associated with good program design and should: \n\n\t\tminimise code duplication & be modularised so components have low coupling and high cohesion. \n\n\t\tuse inheritance, abstract classes and interfaces, as appropriate.\n\n\t\tbe designed to be reusable and easy to maintain.\n\n\t\tbe well documented, displaying both agreed standards of internal documentation and good use of the facilities available in Javadoc.\n\n\t\t\t\n\n\t\tThe project you submit must compile. If your project does not compile, you will not be able to perform in the demo. You must attend the demonstration session. If you do not atten

In [6]:
loader = UnstructuredODTLoader('../../anne_rose_uk/care_part2/Anne_Rose_CARE_Part_2_specification.odt')

In [7]:
odtfile = loader.load()

ModuleNotFoundError: No module named 'exceptions'